<a href="https://colab.research.google.com/github/QuisVenator/AlteredCoal_1.15/blob/master/tesis_kaggle_baseline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Baseline Prediction from Kaggle Dataset

## Setup to allow running locally
We sometimes may want to run this notebook locally or in a managed colab runtime. Here we set it up in a way that allows us to do both. If we set it up on the host such that the local folder also syncs to drive, we can even go back and forth between running locally and remotely. This needs the local runtime to have a folder with the same layout as in the drive.
To run the docker image then:

`docker run --mount type=bind,src="G:/tesis",target=/mnt/tesis -p 127.0.0.1:9000:8080 us-docker.pkg.dev/colab-images/public/runtime`

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  IN_COLAB = True
  DRIVE_PATH = '/content/drive/Othercomputers/My Computer/tesis'
except:
  IN_COLAB = False
  DRIVE_PATH = '/mnt/tesis'

print(IN_COLAB)

False


## First we load datasets from drive and display them

In [ ]:
from IPython.display import display
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
import holidays
import numpy as np
from IPython.display import Markdown, display
import pickle
import lightgbm
from sklearn.model_selection import GroupKFold
from sklearn.metrics import mean_absolute_error
import lightgbm as lgb


def read_data(debug_print=False):
    data: dict[str, pd.DataFrame] = {}
    data['client'] = pd.read_csv(f'{DRIVE_PATH}/datasets/kaggle/client.csv')
    data['electricity_prices'] = pd.read_csv(f'{DRIVE_PATH}/datasets/kaggle/electricity_prices.csv')
    data['forecast_weather'] = pd.read_csv(f'{DRIVE_PATH}/datasets/kaggle/forecast_weather.csv')
    data['gas_prices'] = pd.read_csv(f'{DRIVE_PATH}/datasets/kaggle/gas_prices.csv')
    data['train'] = pd.read_csv(f'{DRIVE_PATH}/datasets/kaggle/train.csv')
    data['historical_weather'] = pd.read_csv(f'{DRIVE_PATH}/datasets/kaggle/historical_weather.csv')
    data['weather_station_to_county_mapping'] = pd.read_csv(f'{DRIVE_PATH}/datasets/kaggle/weather_station_to_county_mapping.csv')

    if debug_print:
        for name, df in data.items():
            print(f'{name}: {df.shape}')
            display(df.head())

    return data['train'], data['client'], data["historical_weather"], data["forecast_weather"], data["gas_prices"], data["electricity_prices"], data["weather_station_to_county_mapping"]


## Data preprocessing
The data we have has a nice readable format for humans, but not computers. For example the dates need to be transformed to cyclical data so the model can understand the seasonality. If we just directly encode months to numbers, december will be seen as very different from january.

Also some data between tables needs to be unified. For the weather forecast gives us latitude and longitude, but our prediction run on counties, so we need to translate positions to counties. Another example is wind direction as degrees in historical weather and as vector in forecast.

Lastly we want to add some features. This includes lagged data points. It is evident that on consecutive days (without other mayor changes between them) data from the first day is a pretty good indicator for behavior on the second day. Another data point we add is wether or not a given day is a holiday or weekend, as data analysis has shown first that weekends show markably different behavior than weekdays and also that holidays behave much more like the former than the later.

In [ ]:
def preprocess_weather_map(df_map):
    df_map = df_map.dropna().copy()
    df_map['latitude'] = df_map['latitude'].astype(float).round(1)
    df_map['longitude'] = df_map['longitude'].astype(float).round(1)
    return df_map

def forecast_map(df, df_map):
    df['latitude'] = df['latitude'].astype(float).round(1)
    df['longitude'] = df['longitude'].astype(float).round(1)

    df = df.merge(df_map[['latitude', 'longitude', 'county']], how='inner', on=['latitude', 'longitude'])
    df = df.dropna(subset=['county'])

    df.rename(columns={'forecast_datetime': 'datetime'}, inplace=True)
    df.drop(columns=['origin_datetime', 'latitude', 'longitude'], inplace=True)

    df = df.sort_values(by='hours_ahead')
    df = df.groupby(['county', 'datetime', 'hours_ahead', 'data_block_id']).mean().reset_index()
    df = df.groupby(['county', 'datetime', 'data_block_id']).first().reset_index()

    return df.add_suffix('_forecast') \
             .rename(columns={'county_forecast': 'county', 'datetime_forecast': 'datetime'})


def historical_map(df, df_map):
    df['latitude'] = df['latitude'].astype(float).round(1)
    df['longitude'] = df['longitude'].astype(float).round(1)

    df = df.merge(df_map[['latitude', 'longitude', 'county']], how='inner', on=['latitude', 'longitude'])
    df = df.dropna(subset=['county'])
    df.drop(columns=['latitude', 'longitude'], inplace=True)

    # CHECKIF: maybe max or min improves?
    return df.groupby(['county', 'datetime']).mean().reset_index().add_suffix('_historical') \
           .rename(columns={'county_historical': 'county'})


def merge_data(df_train, df_client, df_historical_weather, df_forecast_weather, df_gas_prices, df_electricity_prices, df_weather_station_to_county_mapping):
    # Convert coordinates to county codes
    df_weather_station_to_county_mapping = preprocess_weather_map(df_weather_station_to_county_mapping)
    df_historical_weather = historical_map(df_historical_weather, df_weather_station_to_county_mapping).rename(columns={'data_block_id_historical': 'data_block_id'})
    df_forecast_weather = forecast_map(df_forecast_weather, df_weather_station_to_county_mapping).rename(columns={'data_block_id_forecast': 'data_block_id'})


    # Fix date and time for pandas
    df_train['datetime'] = pd.to_datetime(df_train['datetime'])
    df_train['time'] = df_train['datetime'].dt.time
    df_historical_weather['datetime_historical'] = pd.to_datetime(df_historical_weather['datetime_historical'])
    df_historical_weather['time'] = df_historical_weather['datetime_historical'].dt.time
    df_forecast_weather['datetime'] = pd.to_datetime(df_forecast_weather['datetime'])
    df_forecast_weather['datetime'] = df_forecast_weather['datetime'].dt.tz_localize('UTC').dt.tz_convert('Europe/Tallinn').dt.tz_localize(None)
    ## We need to treat the days with time change somehow, so we just drop duplicated hour
    df_forecast_weather = df_forecast_weather.drop_duplicates(subset=['data_block_id', 'datetime', 'county'])

    df_gas_prices.drop(columns=['origin_date', 'forecast_date'], inplace=True)

    df_electricity_prices['time'] = pd.to_datetime(df_electricity_prices['forecast_date']).dt.time
    df_electricity_prices.drop(columns=['origin_date', 'forecast_date'], inplace=True)

    # Merge client data
    df_client.drop(columns=['date'], inplace=True)
    df_client = df_client.groupby(['is_business', 'county', 'product_type', 'data_block_id']).sum().reset_index()
    df_train = pd.merge(df_train, df_client, on=['is_business', 'county', 'product_type', 'data_block_id'], how="left")

    # Merge weather data
    df_train = pd.merge(df_train, df_historical_weather, on=['data_block_id', 'time', 'county'], how='left')
    df_train = pd.merge(df_train, df_forecast_weather, on=['data_block_id', 'datetime', 'county'], how='left')
    # Now go through and replace NaN for counties without weather data with the mean
    weather_nan_cols = df_train.columns[df_train.isna().any()].tolist()
    mean_weather = df_train.groupby('datetime')[weather_nan_cols].transform('mean')
    df_train[weather_nan_cols] = df_train[weather_nan_cols].fillna(mean_weather)

    # Merge price data
    df_train = pd.merge(df_train, df_gas_prices, on=['data_block_id'], how='left')
    df_train = pd.merge(df_train, df_electricity_prices, on=['data_block_id', 'time'], how='left')


    return df_train

def training_specific_merge(df_train):
    new_columns =  []
    for timedelta in range(2,8):
        df_prev_target = df_train[['datetime', 'county', 'target', 'is_consumption', 'is_business', 'product_type']].copy()
        df_prev_target['datetime'] += pd.Timedelta(days=timedelta)
        df_prev_target = df_prev_target.rename(columns={'target': f'target_{timedelta}_ago'})
        new_columns.append(f'target_{timedelta}_ago')

        df_train = pd.merge(df_train, df_prev_target, on=['datetime', 'county', 'is_consumption', 'is_business', 'product_type'], how='left')

    df_train['mean_previous_targets'] = df_train[new_columns].mean(axis=1)

    # Add target over capacity
    df_train['relative_target'] = df_train['target'] / df_train['installed_capacity']

    return df_train

def add_prev_target(df, revealed_targets):
    new_columns =  []
    for timedelta in range(2,8):
        df_prev_target = revealed_targets[['datetime', 'county', 'target', 'is_consumption', 'is_business', 'product_type']].copy()
        df_prev_target['datetime'] += pd.Timedelta(days=timedelta)
        df_prev_target = df_prev_target.rename(columns={'target': f'target_{timedelta}_ago'})
        new_columns.append(f'target_{timedelta}_ago')

        df = pd.merge(df, df_prev_target, on=['datetime', 'county', 'is_consumption', 'is_business', 'product_type'], how='left')

    df['mean_previous_targets'] = df[new_columns].mean(axis=1)

    return df

def transform_data(df):
    df['month'] = df['datetime'].dt.month
    df['weekday'] = df['datetime'].dt.day_of_week
    df['hour'] = df['datetime'].dt.hour

    # Transform cyclical time data
    df['month_sin'] = np.sin(2 * np.pi * df['month'] / 11)
    df['month_cos'] = np.cos(2 * np.pi * df['month'] / 11)
    df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 23)
    df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 23)
    df['weekday_sin'] = np.sin(2 * np.pi * df['weekday'] / 6)
    df['weekday_cos'] = np.cos(2 * np.pi * df['weekday'] / 6)

    ee_holidays = holidays.EE()
    df['is_holiday'] = [1 if date in ee_holidays else 0 for date in df['datetime']]

    df['wind_direction_radians'] = np.radians(df['winddirection_10m_historical'])
    df['10_metre_u_wind_component_historical'] = np.cos(df['wind_direction_radians'])
    df['10_metre_v_wind_component_historical'] = np.sin(df['wind_direction_radians'])
    df = df.drop(columns=['winddirection_10m_historical', 'wind_direction_radians'])

    df = standarize(df, ['target_2_ago', 'target_3_ago', 'target_4_ago', 'target_5_ago', 'target_6_ago', 'target_7_ago', 'surface_solar_radiation_downwards_forecast'])
    df = onehot(df, ['county', 'product_type'])

    return df

standarize_means = {}
standarize_std = {}
def standarize(df, columns):
    for col in columns:
        col_mean = df[col].mean()
        col_std = df[col].std()
        standarize_means[col] = col_mean
        standarize_std[col] = col_std
        if col_std != 0:
            df.loc[:, col] = df[col].apply(lambda x: (x - col_mean) / col_std)
        else:
            df.loc[:, col] = df[col].apply(lambda x: 0)
    return df

def onehot(df, columns):
    for column in columns:
        one_hot = pd.get_dummies(df[column], prefix=column)
        df = pd.concat([df, one_hot], axis=1)
    df = df.drop(columns=columns)
    return df

# Debugging only function
def check_duplicates(df, stage):
    duplicates = df.duplicated('row_id', keep=False)
    if duplicates.any():
        dup_rows = df[duplicates]
        sorted_dup = dup_rows.sort_values(by='row_id')
        display(sorted_dup)
        raise ValueError(f"row_id contains duplicates in stage {stage}!")

## Defining features for production and consumption
We decide to separate production and consumption predictions. The reason for this is that data analysis shows that some features can lead to improvements in one but not the other. Probably the easiest example that makes intuitive sense are lagged targets. For production in most cases we would assume that it is the same on days with identical weather conditions. For consumption on the other hand we would assume that there are factors that can not be captured by the features that will affect the consumption, even on days with exactly equal weather conditions. A possible scenario could be a holiday season, when maybe people visit more a specific county. If we include lagged consumption data and see that for the past n days the consumption was very high or low, it is reasonably to assume that this behavior persists until proven otherwise.

In [ ]:
features_con = [
    'target_2_ago',
    'target_3_ago',
    'target_4_ago',
    'target_5_ago',
    'target_6_ago',
    'target_7_ago',
    'temperature_forecast',
    'dewpoint_forecast',
    'cloudcover_high_forecast',
    'cloudcover_mid_forecast',
    'cloudcover_low_forecast',
    'cloudcover_total_forecast',
    '10_metre_u_wind_component_forecast',
    '10_metre_v_wind_component_forecast',
    'direct_solar_radiation_forecast',
    'surface_solar_radiation_downwards_forecast',
    'temperature_historical',
    'dewpoint_historical',
    'rain_historical',
    'snowfall_historical',
    'surface_pressure_historical',
    'cloudcover_high_historical',
    'cloudcover_mid_historical',
    'cloudcover_low_historical',
    'cloudcover_total_historical',
    '10_metre_u_wind_component_historical',
    '10_metre_v_wind_component_historical',
    'shortwave_radiation_historical',
    'direct_solar_radiation_historical',
    'diffuse_radiation_historical',
    'lowest_price_per_mwh',
    'highest_price_per_mwh',
    'euros_per_mwh',
    'eic_count',
    'installed_capacity',
    'product_type_0',
    'product_type_1',
    'product_type_2',
    'product_type_3',
    'is_business',
    'county_0',
    'county_1',
    'county_2',
    'county_3',
    'county_4',
    'county_5',
    'county_6',
    'county_7',
    'county_8',
    'county_9',
    'county_10',
    'county_11',
    'county_12',
    'county_13',
    'county_14',
    'county_15',
    'month_sin',
    'month_cos',
    'weekday_sin',
    'weekday_cos',
    'hour_sin',
    'hour_cos',
    'is_holiday',
    'mean_previous_targets'
]

features_prod = [
    'target_2_ago',
    'target_3_ago',
    'target_4_ago',
    'target_5_ago',
    'target_6_ago',
    'target_7_ago',
    'temperature_forecast',
    'dewpoint_forecast',
    'cloudcover_high_forecast',
    'cloudcover_mid_forecast',
    'cloudcover_low_forecast',
    'cloudcover_total_forecast',
    '10_metre_u_wind_component_forecast',
    '10_metre_v_wind_component_forecast',
    'direct_solar_radiation_forecast',
    'surface_solar_radiation_downwards_forecast',
    'temperature_historical',
    'dewpoint_historical',
    'rain_historical',
    'snowfall_historical',
    'surface_pressure_historical',
    'cloudcover_high_historical',
    'cloudcover_mid_historical',
    'cloudcover_low_historical',
    'cloudcover_total_historical',
    '10_metre_u_wind_component_historical',
    '10_metre_v_wind_component_historical',
    'shortwave_radiation_historical',
    'direct_solar_radiation_historical',
    'diffuse_radiation_historical',
    'lowest_price_per_mwh',
    'highest_price_per_mwh',
    'euros_per_mwh',
    'eic_count',
    'installed_capacity',
    'product_type_0',
    'product_type_1',
    'product_type_2',
    'product_type_3',
    'is_business',
    'county_0',
    'county_1',
    'county_2',
    'county_3',
    'county_4',
    'county_5',
    'county_6',
    'county_7',
    'county_8',
    'county_9',
    'county_10',
    'county_11',
    'county_12',
    'county_13',
    'county_14',
    'county_15',
    'month_sin',
    'month_cos',
    'weekday_sin',
    'weekday_cos',
    'hour_sin',
    'hour_cos',
    'is_holiday'
]

## Train function
Lastly we define the function that will make our predictions. The parameters for our model here are already result of hyperparameter optimizations that have been run separately.

In [ ]:
def train():
    df_train, df_client, df_historical_weather, df_forecast_weather, df_gas_prices, df_electricity_prices, df_weather_station_to_county_mapping = read_data()

#     df_train = df_train[df_train["datetime"] > '2021-12-31']

    df = merge_data(df_train.copy(), df_client, df_historical_weather, df_forecast_weather, df_gas_prices, df_electricity_prices, df_weather_station_to_county_mapping)
    df = training_specific_merge(df)
    df = transform_data(df).dropna()

    df_consumption = df[df['is_consumption'] == 1]
    df_production = df[df['is_consumption'] == 0]

    models = {
        "consumption": {
            "data": df_consumption,
            "models": [],
            "mae": 0,
            # From 5 fold cross validation (grouped by data_block_id), 100 iteration random search parameter optimization. MAE: 55.84955665117576
            "params": [
                {
                    "num_leaves": 49,
                    "min_child_samples": 87,
                    "min_child_weight": 0.02057785716550018,
                    "subsample": 0.794696861183782,
                    "colsample_bytree": 0.9624395150874216,
                    "reg_alpha": 0.8687887310208573,
                    "reg_lambda": 0.7001568153893514,
                    "learning_rate": 0.139020672406113,
                    "n_estimators": 794
                },
                {
                    "num_leaves": 47,
                    "min_child_samples": 237,
                    "min_child_weight": 0.008327236865873834,
                    "subsample": 0.7824279936868144,
                    "colsample_bytree": 0.9140703845572055,
                    "reg_alpha": 0.39934756431671947,
                    "reg_lambda": 1.0284688768272232,
                    "learning_rate": 0.1284829137724085,
                    "n_estimators": 796
                },
                {
                    "num_leaves": 49,
                    "min_child_samples": 276,
                    "min_child_weight": 0.008445655331234862,
                    "subsample": 0.9760533769831113,
                    "colsample_bytree": 0.989465534702127,
                    "reg_alpha": 0.5678419494749314,
                    "reg_lambda": 0.6107277206887869,
                    "learning_rate": 0.10712275071724532,
                    "n_estimators": 787
                },
                {
                    "num_leaves": 47,
                    "min_child_samples": 70,
                    "min_child_weight": 0.013349630192554331,
                    "subsample": 0.8446612641953124,
                    "colsample_bytree": 0.6028265220878869,
                    "reg_alpha": 0.046124850082831514,
                    "reg_lambda": 1.0495493205167783,
                    "learning_rate": 0.08997219434305109,
                    "n_estimators": 800
                }
            ],
            "booster_weight": [
                0.25,
                0.25,
                0.25,
                0.25,
            ],
            "features": features_con
        },
        "production": {
            "data": df_production,
            "models": [],
            "mae": 0,
            # Same as for consumption
            "params": [
                {
                    "num_leaves": 43,
                    "min_child_samples": 166,
                    "min_child_weight": 0.013022300234864177,
                    "subsample": 0.8832290311184181,
                    "colsample_bytree": 0.608233797718321,
                    "reg_alpha": 1.9398197043239886,
                    "reg_lambda": 1.6648852816008435,
                    "learning_rate": 0.052467822135655234,
                    "n_estimators": 558
                },
                {
                    "num_leaves": 47,
                    "min_child_samples": 70,
                    "min_child_weight": 0.013349630192554331,
                    "subsample": 0.8446612641953124,
                    "colsample_bytree": 0.6028265220878869,
                    "reg_alpha": 0.046124850082831514,
                    "reg_lambda": 1.0495493205167783,
                    "learning_rate": 0.08997219434305109,
                    "n_estimators": 442
                },
                {
                    "num_leaves": 41,
                    "min_child_samples": 107,
                    "min_child_weight": 0.018208092366233507,
                    "subsample": 0.7001005442063456,
                    "colsample_bytree": 0.6155338937717693,
                    "reg_alpha": 0.6065310293464456,
                    "reg_lambda": 1.0741648543933109,
                    "learning_rate": 0.07533024835920818,
                    "n_estimators": 510
                },
                {
                    "num_leaves": 46,
                    "min_child_samples": 167,
                    "min_child_weight": 0.005375284391461405,
                    "subsample": 0.8232408008069365,
                    "colsample_bytree": 0.7615344684232164,
                    "reg_alpha": 0.12978449421796312,
                    "reg_lambda": 0.5078308278686894,
                    "learning_rate": 0.05937521256772024,
                    "n_estimators": 487
                }
            ],
            "booster_weight": [
                0.25,
                0.25,
                0.25,
                0.25,
            ],
            "features": features_prod
        }
    }

    for model_name, model_info in models.items():
        df = model_info['data']
        df.reset_index(drop=True, inplace=True)

#         with pd.option_context('display.max_columns', None):
#             display(df[['row_id', 'datetime', *features]])


#         df_final = df[df['datetime'] <= '2023-01-31']
#         df_test = df[df['datetime'] > '2023-01-31']
#         df_final = df[df['datetime'].dt.month.isin([6, 7, 8, 9, 10])]
#         df_test = df[df['datetime'].dt.month.isin([6, 7, 8, 9, 10])]
        df_final = df
        df_test = df


        final_pred = 0
        num_boosters = len(model_info['params'])
        for i in range(num_boosters):
            lgb_dataset = lgb.Dataset(df_final[model_info["features"]], df_final['target'])
            params = {
                'objective': 'regression',
                'metric': 'mae',
                'verbose': -1
            }
            params.update(model_info['params'][i])
            model_info['models'].append(lgb.train(params, lgb_dataset))
            pred = model_info['models'][i].predict(df_test[model_info["features"]], num_iteration=model_info['models'][i].best_iteration)

            final_pred = model_info['booster_weight'][i] * pred + final_pred

        mae = mean_absolute_error(df_test['target'], final_pred)
        display(Markdown(f'# THE MAE FOR ACTUAL MODEL IS: {mae}'))
        display(df['row_id'])
        display(final_pred)

    return models['production'], models['consumption']

model_production, model_consumption = train()

/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


# THE MAE FOR ACTUAL MODEL IS: 31.43669408189782

,row_id
0,20497
1,20499
2,20501
3,20503
4,20505
...,...
992329,2018343
992330,2018345
992331,2018347
992332,2018349


array([106.58613594,  19.16398855, 720.9798309 , ..., 308.78611598,
        36.61500232, 185.33841987])

/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
/usr/local/lib/python3.11/dist-packages/lightgbm/engine.py:204: UserWarning: Found `n_estimators` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


# THE MAE FOR ACTUAL MODEL IS: 18.856702120500177

,row_id
0,20496
1,20498
2,20500
3,20502
4,20504
...,...
992329,2018342
992330,2018344
992331,2018346
992332,2018348


array([-1.59926169, -2.22795783,  0.29914659, ..., -1.12334802,
       -1.04011274,  1.27806294])

## Saving the models
Now we will save the models we obtained so we can later re-use them easely

In [ ]:
for i, m in enumerate(model_production['models']):
    m.save_model(f"{DRIVE_PATH}/models/kaggle_best_submission/improved/model_production_{i}.txt")

# Write obtained mae to textfile
with open(f'{DRIVE_PATH}/models/kaggle_best_submission/improved/mae_production.txt', 'w') as f:
    f.write(str(model_production['mae']))

for i, m in enumerate(model_consumption['models']):
    m.save_model(f"{DRIVE_PATH}/models/kaggle_best_submission/improved/model_consumption_{i}.txt")

with open(f'{DRIVE_PATH}/models/kaggle_best_submission/improved/mae_consumption.txt', 'w') as f:
    f.write(str(model_consumption['mae']))
